pip install markitdown   
This is an open-source package for converting PDF to MarkDown.

In [4]:
from markitdown import MarkItDown

md = MarkItDown()
result = md.convert("data/2024全球经济金融展望报告.pdf")
with open("data/md_file.md", "w", encoding="utf-8") as f:
    f.write(result.text_content)

Merge all page content into one string

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.schema import Document
import re

loader = PyPDFLoader("data/2024全球经济金融展望报告.pdf")
documents = loader.load()

### documents is a list where each element is a Document object, which respresenting each page of content loaded from  the PDF document.
pattern = r"^全球经济金融展望报告\n中国银行研究院 \d+ 2024年"
merged_docs = [Document(page_content='\n'.join(re.sub(pattern, '', doc.page_content) for doc in documents))]


split document

In [11]:
import os
import pickle
from uuid import uuid4
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents, filepath, chunk_size=400, chunk_overlap=40, seperators=['\n\n\n', '\n\n'], force_split=False):
    if os.path.exists(filepath) and not force_split:
        print('found cache, restoring...')
        return pickle.load(open(filepath, 'rb'))

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperators
    )
    split_docs = splitter.split_documents(documents)
    for chunk in split_docs:
        chunk.metadata['uuid'] = str(uuid4())

    pickle.dump(split_docs, open(filepath, 'wb'))

    return split_docs


In [13]:
splitted_docs = split_docs(documents, os.path.join("outputs", 'split_docs.pkl'), chunk_size=500, chunk_overlap=50)
splitted_docs_large = split_docs(merged_docs, os.path.join("outputs", 'split_docs_large.pkl'), chunk_size=1500, chunk_overlap=100)

extract qa

In [15]:
qa_gen_prompt_tmpl = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成8个问题、用户回答这个问题的上下文，和基于上下文对问题的回答。

对问题、上下文、答案的要求：

问题要与这段文本相关，不要询问类似“这个问题的答案在哪一章”这样的问题
上下文：上下文必须与原始文本的内容保持一致，不要进行缩写、扩写、改写、摘要、替换词语等
答案：回答请保持完整且简洁，无须重复问题。答案要能够独立回答问题，而不是引用现有的章节、页码等

返回结果以JSON形式组织，格式为[{"question": "...", "context": ..., "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

qa_gen_prompt_tmpl_large_context = """
我会给你一段文本（<document></document>之间的部分），你需要阅读这段文本，分别针对这段文本生成2个问题，和基于这段文本对问题的回答，回答请保持完整，无须重复问题。
尽可能创建一些需要综合*大段*文本才能回答的问题，但不要问类似“这一段主要讲了什么内容”这样的问题，答案要能够独立回答问题，而不是引用现有的章节、页码等；不要问具体过于细节的问题，例如“海湾国家的2024年预期经济增长率是多少”，而是尽可能问类似“2024年全球经济的几大趋势是什么”、“受局部中东地区紧张局势影响，可能对全球原物料有哪些影响”。
返回结果以JSON形式组织，格式为[{"question": "...", "answer": "..."}, ...]。
如果当前文本主要是目录，或者是一些人名、地址、电子邮箱等没有办法生成有意义的问题时，可以返回[]。

下方是文本：
<document>
{{document}}
</document>

请生成结果：
"""

In [32]:
from openai import OpenAI
import time
import random

from zhipuai import ZhipuAI
client = ZhipuAI(api_key="")  
# response = client.chat.completions.create(
#     model="glm-4-plus",  
#     messages=[
#         {"role": "system", "content": "你是一个乐于回答各种问题的小助手，你的任务是提供专业、准确、有洞察力的建议。"},
#         {"role": "user", "content": "你是谁"},
#     ],
#     stream=True,
# )
# for chunk in response:
#     print(chunk.choices[0].delta.content)

def build_qa_prompt(prompt_tmpl, text):
    prompt = prompt_tmpl.replace('{{document}}', text).strip()
    return prompt

def chat(prompt, max_retry=3, debug=False, top_p=0.95, temperature=0.85):
    def do_chat(prompt):
        completion = client.chat.completions.create(
            model="glm-4-plus",
            messages=[
                {"role": "system", "content": "你是一个乐于回答各种问题的小助手，你的任务是提供专业、准确、有洞察力的建议。"},
                {"role": "user", "content": prompt}
            ],
            top_p=top_p,
            temperature=temperature
        )
        return completion.choices[0].message.content

    while max_retry > 0:
        try:
            return do_chat(prompt)
        except Exception as e:
            max_retry -= 1
            sleep_seconds = random.randint(1, 4)
            if debug:
                print(f"{str(e)}, remain retry: {max_retry}, sleeping {sleep_seconds}s {prompt}")
            time.sleep(sleep_seconds)
    return None

In [24]:
import pandas as pd
import re
import io
import json

# 读取Markdown文件
with open('data/6. 特高压直流测量设备关键点技术监督实施细则.md', 'r', encoding='utf-8') as file:
    markdown_text = file.read()

# 提取表格部分
table_pattern = re.compile(r'\|.*\|\n\|.*\|\n(\|.*\|\n)*')  # 匹配Markdown表格
table_match = table_pattern.search(markdown_text)

if table_match:
    table_text = table_match.group(0)
    # 将Markdown表格转换为Pandas DataFrame
    df = pd.read_csv(io.StringIO(table_text), sep='|', header=0, skipinitialspace=True)
    df = df.dropna(axis=1, how='all')  # 删除空列
    df.columns = df.columns.str.strip()  # 清理列名

    # 将每一行提取为JSON对象
    with open('output.jsonl', 'w', encoding='utf-8') as json_file:
        for _, row in df.iterrows():
            row_dict = row.to_dict()
            json_file.write(json.dumps(row_dict, ensure_ascii=False) + '\n')

    print("数据已保存到 output.jsonl 文件中。")
else:
    print("未找到Markdown表格。")

数据已保存到 output.jsonl 文件中。


In [25]:
import json

# 定义键名称映射
key_mapping = {
    "特高压直流测量设备关键点技术监督实施细则": "特高压直流测量设备关键点技术监督实施细则 序号",
    "工程名称": "监督项目",
    "Unnamed: 3": "监督标准",
    "生产厂家": "监督性质",
    "Unnamed: 5": "监督方式",
    "Unnamed: 6": "监督出处",
    "Unnamed: 7": "监督结论",
    "Unnamed: 8": "监督单位",
    "Unnamed: 9": "问题说明"
}

# 读取原始JSONL文件
input_file = 'output.jsonl'
output_file = 'modified_output.jsonl'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        # 解析JSON对象
        data = json.loads(line.strip())
        
        # 修改键名称
        modified_data = {key_mapping.get(k, k): v for k, v in data.items()}
        
        # 写入新的JSONL文件
        outfile.write(json.dumps(modified_data, ensure_ascii=False) + '\n')

print(f"修改后的数据已保存到 {output_file} 文件中。")

修改后的数据已保存到 modified_output.jsonl 文件中。


In [23]:
import json

def process_jsonl(input_file, output_file):
    # 读取JSONL文件
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    # 解析JSONL文件
    json_list = [json.loads(line) for line in lines]

    # 创建一个字典来存储每个主序号的"监督项目"值
    main_supervision_dict = {}

    # 第一次遍历：收集所有主序号的"监督项目"值
    for json_obj in json_list:
        key = json_obj.get("测量设备基础信息 序号")
        if isinstance(key, str):
            # 提取主序号（如 "1.1" 的主序号是 "1"）
            main_key = key.split('.')[0]
            if '.' not in key:  # 如果当前是主序号（如 "1", "2" 等）
                main_supervision_dict[main_key] = json_obj.get("监督项目", "")

    # 第二次遍历：更新所有子序号的"监督项目"值
    for json_obj in json_list:
        key = json_obj.get("测量设备基础信息 序号")
        if isinstance(key, str):
            # 提取主序号
            main_key = key.split('.')[0]
            if '.' in key:  # 如果当前是子序号（如 "1.1", "2.1" 等）
                main_supervision = main_supervision_dict.get(main_key, "")
                json_obj["监督项目"] = main_supervision + json_obj.get("监督项目", "")

    # 将处理后的JSON对象写回到新的JSONL文件
    with open(output_file, 'w', encoding='utf-8') as outfile:
        for json_obj in json_list:
            outfile.write(json.dumps(json_obj, ensure_ascii=False) + '\n')

# 示例调用
input_file = 'modified_output.jsonl'
output_file = 'modified_output_.jsonl'
process_jsonl(input_file, output_file)